# Classification of data as per IEC 60599:2022

### This notebook is used to classify the input data as per the latest IEC60599:2022 Standard

In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Mounting the gdrive to access input files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
def get_prediction(row):
    """Predicts the transformer faults as per IEC 60599 ratios"""
    if row["CH4/H2"] < 0.1 and row["C2H4/C2H6"] < 0.2:
        row["Fault"] = row["Fault"] + "PD"
    if row["C2H2/C2H4"] > 1 and row["CH4/H2"] > 0.1 and row["CH4/H2"] < 0.5 and row["C2H4/C2H6"] > 1:
        row["Fault"] = row["Fault"] + "D1" 
    if row["C2H2/C2H4"] > 0.6 and row["C2H2/C2H4"] < 2.5 and row["CH4/H2"] > 0.1 and row["CH4/H2"] < 1 and row["C2H4/C2H6"] > 2:
        row["Fault"] = row["Fault"] + "D2"
    if row["C2H4/C2H6"] < 1:
        row["Fault"] = row["Fault"] + "T1"
    if row["C2H2/C2H4"] < 0.1 and row["CH4/H2"] > 1 and row["C2H4/C2H6"] > 1 and row["C2H4/C2H6"] < 4:
        row["Fault"] = row["Fault"] + "T2"
    if row["C2H2/C2H4"] < 0.2 and row["CH4/H2"] > 1 and row["C2H4/C2H6"] > 4:
        row["Fault"] = row["Fault"] + "T3"
    if row["Fault"] == '':
        row["Fault"] = row["Fault"] + "NF"
    return row    

In [4]:
def get_classification(df):
    """ Calculates the three ratios and calls the function to predict faults """
    df["CH4/H2"] = df["CH4"]/df["H2"]
    df["C2H2/C2H4"] = df["C2H2"]/df["C2H4"]
    df["C2H4/C2H6"] = df["C2H4"]/df["C2H6"]
    df.loc[~np.isfinite(df['CH4/H2']), 'CH4/H2'] = 0
    df.loc[~np.isfinite(df['C2H2/C2H4']), 'C2H2/C2H4'] = 0
    df.loc[~np.isfinite(df['C2H4/C2H6']), 'C2H4/C2H6'] = 0
    for row in range(0, len(df.index)):
        df.iloc[row,:] = get_prediction(df.iloc[row,:])
    return df

In [5]:
def write_csv(df, name):
    """To write a csv file with given name"""
    df.to_csv("/content/gdrive/My Drive/Colab Notebooks/IEC60599_2022_Classification_Output/" + name + ".csv", index=False) 

In [7]:
#Load the csv file
filenames = ["Consolidated_Actual_Fault_Data.csv"]
out_dfs = []
for filename in filenames:
  df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/" + filename)
  print("Displaying the dataframe ", filename, " before prediction")
  df = df.iloc[:,:5]
  df["Fault"] = ""
  display(df)
  out_dfs.append(get_classification(df))
  print("Displaying the dataframe ", filename, " after prediction")
  display(out_dfs[-1])
print("Writing the created dataframes into csv files:")
write_csv(out_dfs[0],"IEC60599_2022_Actual_Data")

Displaying the dataframe  Consolidated_Actual_Fault_Data.csv  before prediction


,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,
1,46.13,11.57,0.63,33.14,8.52,
2,33.66,2.97,2.48,33.17,27.72,
3,143.80,21.53,0.20,8.21,2.71,
4,170.00,300.00,8.90,580.00,44.00,
...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,
469,164.00,244.00,103.00,497.00,8.30,
470,135.65,278.53,58.86,492.00,2.95,
471,135.88,362.42,125.22,826.65,3.74,


Displaying the dataframe  Consolidated_Actual_Fault_Data.csv  after prediction


,H2,CH4,C2H2,C2H4,C2H6,Fault,CH4/H2,C2H2/C2H4,C2H4/C2H6
0,80.00,10.00,0.00,4.00,1.50,NF,0.125000,0.000000,2.666667
1,46.13,11.57,0.63,33.14,8.52,NF,0.250813,0.019010,3.889671
2,33.66,2.97,2.48,33.17,27.72,NF,0.088235,0.074766,1.196609
3,143.80,21.53,0.20,8.21,2.71,NF,0.149722,0.024361,3.029520
4,170.00,300.00,8.90,580.00,44.00,T3,1.764706,0.015345,13.181818
...,...,...,...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,T3,1.454836,0.119177,38.028825
469,164.00,244.00,103.00,497.00,8.30,NF,1.487805,0.207243,59.879518
470,135.65,278.53,58.86,492.00,2.95,T3,2.053299,0.119634,166.779661
471,135.88,362.42,125.22,826.65,3.74,T3,2.667206,0.151479,221.029412


Writing the created dataframes into csv files:
